---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import datetime as dt #### need this ilater


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
data = pd.read_csv("City_Zhvi_AllHomes.csv")
university_towns = pd.read_table("university_towns.txt", header = None)

In [3]:
university_towns.head()

,0
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]


In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
#### Get state and towns list by adding the assging state variable in loop then add it together with town.
unclean_states = list()
unclean_towns = list()
for string in list(university_towns[0]):
    if string.find("[edit]") >= 0:
        state = string
    if string.find("[edit]") == -1:
        unclean_towns.append(string)
        unclean_states.append(state)
### Clean state by sliting and taking the 1st item
clean_states = list()
for state in unclean_states:
    clean_state = state.split("[")[0]
    clean_states.append(clean_state)
clean_towns = list()
for town in unclean_towns:
    clean_town = town.split(" (")[0]
    #clean_town = clean_town.split(", ")[0]
    #clean_town = clean_town.split("-")[0]
    clean_towns.append(clean_town)

### The dict is reverese do i reveres the item wnad keys to get the result i wanted
## https://stackoverflow.com/questions/21546739/load-data-from-txt-with-pandas
inv_state = {v: k for k, v in states.items()}
df_dict = {"State":clean_states,
       "RegionName" :clean_towns}
Uni_town_df = pd.DataFrame(df_dict)
Uni_town_df.replace({ "State" : inv_state}, inplace = True)
Uni_town_df.shape

(517, 2)

In [6]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [7]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    # Use this dictionary to map state names to two letter acronyms
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    #### Get state and towns list by adding the assging state variable in loop then add it together with town.
    unclean_states = list()
    unclean_towns = list()
    for string in list(university_towns[0]):
        if string.find("[edit]") >= 0:
            state = string
        if string.find("[edit]") == -1:
            unclean_towns.append(string)
            unclean_states.append(state)
    ### Clean state by sliting and taking the 1st item
    clean_states = list()
    for state in unclean_states:
        clean_state = state.split("[")[0]#.split('\n')[0] ### add line
        clean_states.append(clean_state)
    clean_towns = list()
    for town in unclean_towns:
        clean_town = town.split(" (")[0]#.split('\n')[0] ### add line
        #clean_town = clean_town.split(", ")[0]
        #clean_town = clean_town.split("-")[0]
        
        clean_towns.append(clean_town)

    ### The dict is reverese do i reveres the item wnad keys to get the result i wanted
    ## https://stackoverflow.com/questions/21546739/load-data-from-txt-with-pandas
    #inv_state = {v: k for k, v in states.items()}
    df_dict = {"State":clean_states,
           "RegionName" :clean_towns}
    Uni_town_df = pd.DataFrame(df_dict)
    Uni_town_df.replace({ "State" : states}, inplace = True)
    return Uni_town_df[["State","RegionName"]]

In [8]:
GDP = pd.ExcelFile("gdplev.xls")
GDP = GDP.parse(GDP.sheet_names[0])
GDP.columns = ['Current-Dollar and "Real" Gross Domestic Product ' ,"GDP in billions of current dollars nan" , "GDP in billions of chained 2009 dollars",
              "no need", "Quarterly" ,"GDP in billions of current dollars","GDP in billions of chained 2009 dollars", "no need"]
GDP = GDP[7:][["Quarterly","GDP in billions of current dollars"]]


range_panda= range(0,len(GDP["GDP in billions of current dollars"]),1)
recession = list()
for num in list(range_panda):
    if num == len(GDP["GDP in billions of current dollars"])-4:
        break
    value  = GDP["GDP in billions of current dollars"].iloc[num]
    value2 = GDP["GDP in billions of current dollars"].iloc[num + 1]
    value3 = GDP["GDP in billions of current dollars"].iloc[num + 2]
    value4 = GDP["GDP in billions of current dollars"].iloc[num + 3]
    value5 = GDP["GDP in billions of current dollars"].iloc[num + 4]

    if (value > value2> value3) and (value3<value4<value5) :
        recession.append(num +1)
A = GDP["Quarterly"].iloc[recession[0]]
A

'1953q4'

In [9]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.ExcelFile("gdplev.xls")
    GDP = GDP.parse(GDP.sheet_names[0])
    GDP.columns = ['Current-Dollar and "Real" Gross Domestic Product ' ,"GDP in billions of current dollars nan" , "GDP in billions of chained 2009 dollars",
                  "no need", "Quarterly" ,"GDP in billions of current dollars","GDP in billions of chained 2009 dollars", "no need"]
    ### Only use these values
    GDP = GDP[7:][["Quarterly","GDP in billions of current dollars","GDP in billions of chained 2009 dollars"]]
    ## only 2000 data
    GDP = GDP[212:]
    ## choses the columns becuasue you might change it
    col = "GDP in billions of current dollars"
    ## use for loop by range to get the number of indecies
    range_panda= range(0,len(GDP[col]),1)
    recession = list()
    for num in list(range_panda):
        if num == len(GDP[col])-4:
            break
        value  = GDP[col].iloc[num]
        value2 = GDP[col].iloc[num + 1]
        value3 = GDP[col].iloc[num + 2]
        value4 = GDP[col].iloc[num + 3]
        value5 = GDP[col].iloc[num + 4]

        if (value > value2> value3) and (value3<value4<value5) :
            recession.append(num)
   
    A = GDP["Quarterly"].iloc[recession[0]-1] ## put minus 1 to specify when it has started
     
    return A

In [10]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP = pd.ExcelFile("gdplev.xls")
    GDP = GDP.parse(GDP.sheet_names[0])
    GDP.columns = ['Current-Dollar and "Real" Gross Domestic Product ' ,"GDP in billions of current dollars nan" , "GDP in billions of chained 2009 dollars",
                  "no need", "Quarterly" ,"GDP in billions of current dollars","GDP in billions of chained 2009 dollars", "no need"]
    GDP = GDP[7:][["Quarterly","GDP in billions of current dollars","GDP in billions of chained 2009 dollars"]]
    GDP = GDP[212:]
    col = "GDP in billions of current dollars"
    range_panda= range(0,len(GDP[col]),1)
    recession = list()
    for num in list(range_panda):
        if num == len(GDP[col])-4:
            break
        value  = GDP[col].iloc[num]
        value2 = GDP[col].iloc[num + 1]
        value3 = GDP[col].iloc[num + 2]
        value4 = GDP[col].iloc[num + 3]
        value5 = GDP[col].iloc[num + 4]

        if (value > value2> value3) and (value3<value4<value5) :
            recession.append(num)
   
    A = GDP["Quarterly"].iloc[recession[0]+4] ### Last recession value
    
    return A

In [11]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = pd.ExcelFile("gdplev.xls")
    GDP = GDP.parse(GDP.sheet_names[0])
    GDP.columns = ['Current-Dollar and "Real" Gross Domestic Product ' ,"GDP in billions of current dollars nan" , "GDP in billions of chained 2009 dollars",
                  "no need", "Quarterly" ,"GDP in billions of current dollars","GDP in billions of chained 2009 dollars", "no need"]
    GDP = GDP[7:][["Quarterly","GDP in billions of current dollars","GDP in billions of chained 2009 dollars"]]
    GDP = GDP[212:]
    col = "GDP in billions of current dollars"
    range_panda= range(0,len(GDP[col]),1)
    recession = list()
    for num in list(range_panda):
        if num == len(GDP[col])-4:
            break
        value  = GDP[col].iloc[num]
        value2 = GDP[col].iloc[num + 1]
        value3 = GDP[col].iloc[num + 2]
        value4 = GDP[col].iloc[num + 3]
        value5 = GDP[col].iloc[num + 4]

        if (value > value2> value3) and (value3<value4<value5) :
            recession.append(num)
   
    A = GDP["Quarterly"].iloc[recession[0]+2] ### Last recession value
    
    return A

### read the data
housing = pd.read_csv("City_Zhvi_AllHomes.csv")
years_col = list(housing.columns)[6:] ## The all years col
cols = years_col[45:] ### from 2000

#housing.rename({"State" : states} , inplace = True)
housing["State"] = housing["State"].map(states)
housing.set_index(["State", "RegionName"], inplace= True)
housing = housing[cols]
### Change the cols names to quarters of each year
housing_q_means = list()
for col in housing.columns:
    col_date = col.split("-")
    quarter = pd.Timestamp(dt.date(int(col_date[0]), int(col_date[1]) , 1)).quarter
    year  = pd.Timestamp(dt.date(int(col_date[0]), int(col_date[1]) , 1)).year
    year_q = str(year)+"q"+str(quarter)
    housing_q_means.append(year_q)

housing.columns = housing_q_means ### change the origin
#### Create q mean cols and have all means in it
quarter_means_cols = list()
for col in set(housing_q_means):
    housing[col + "mean"] = housing[col].mean(axis = 1)
    quarter_means_cols.append(col + "mean")
quarter_means_cols.sort()
housing = housing[quarter_means_cols]
 ## now change means
quarter_means_cols2 = list()
for string in quarter_means_cols:
    quarter_means_cols2.append(string.split("mean")[0])
housing.columns = quarter_means_cols2

housing.head()


In [12]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    ### read the data
    housing = pd.read_csv("City_Zhvi_AllHomes.csv")
    years_col = list(housing.columns)[6:] ## The all years col
    cols = years_col[45:] ### from 2000

    #housing.rename({"State" : states} , inplace = True)
    housing["State"] = housing["State"].map(states)  ### $$$ you should have maped the state with the dict given above
    housing.set_index(["State", "RegionName"], inplace= True)
    housing = housing[cols]
    ### Change the cols names to quarters of each year
    housing_q_means = list()
    for col in housing.columns:
        col_date = col.split("-")
        quarter = pd.Timestamp(dt.date(int(col_date[0]), int(col_date[1]) , 1)).quarter
        year  = pd.Timestamp(dt.date(int(col_date[0]), int(col_date[1]) , 1)).year
        year_q = str(year)+"q"+str(quarter)
        housing_q_means.append(year_q)

    housing.columns = housing_q_means ### change the origin
    #### Create q mean cols and have all means in it
    quarter_means_cols = list()
    for col in set(housing_q_means):
        housing[col + "mean"] = housing[col].mean(axis = 1)
        quarter_means_cols.append(col + "mean")
    quarter_means_cols.sort()
    housing = housing[quarter_means_cols]
     ## now change means
    quarter_means_cols2 = list()
    for string in quarter_means_cols:
        quarter_means_cols2.append(string.split("mean")[0])
    housing.columns = quarter_means_cols2



    return housing

In [17]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 

    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    
    '''
    Hypothesis: University towns have their mean housing prices less effected by recessions.
    Run a t-test to compare the ratio of the mean price of houses in university towns the quarter
    before the recession starts compared to the recession bottom. 
    (price_ratio=quarter_before_recession/recession_bottom)
    '''
    
    ## Get the data
    housing = convert_housing_data_to_quarters()
    housing = housing.reset_index()
    ## Create Ratio of resssion
    housing["ratio"] = housing[get_recession_start()] / housing[get_recession_bottom()]
    unitowns = get_list_of_university_towns()

    ## Merge with town
    unitowns_housing = pd.merge(housing,unitowns,how='inner',on=['State','RegionName']) ### inner section
    ## assign data on the data now
    unitowns_housing['unitown'] = 0
    unitowns_housing_all = pd.merge(housing,unitowns_housing,how='outer',on=['State','RegionName',
                                                                             get_recession_start(),
                                                                             get_recession_bottom(),
                                                                             'ratio']) ## outter everything other than inner

    unitowns_housing_all['unitown'] = unitowns_housing_all['unitown'].fillna(1)
    unitown_data = unitowns_housing_all[unitowns_housing_all['unitown'] == 0]['ratio'].dropna()
    not_unitown_data = unitowns_housing_all[unitowns_housing_all['unitown'] == 1]['ratio'].dropna()

    pval = ttest_ind(unitown_data, not_unitown_data) 


    ## Better assign better
    better = None
    if unitown_data.mean() < not_unitown_data.mean(): #lower mean price ratio
        better = "university town"
    if unitown_data.mean() > not_unitown_data.mean():
        better = "non-university town"

    ## different assign value   
    different = False
    if pval[1] <0.01:
        different= True


    return (different, pval[1] , better)

In [16]:
run_ttest()

(True, 0.0054964273536938875, 'non-university town')